# 네이버 영화 수집 긍/부정 모델

## 1. 네이버 영화에서 몇개의 영화를 선택하여 한줄평 text 수집한다.

In [51]:
import requests
from  bs4 import BeautifulSoup
import pandas as pd
from konlpy.tag import Okt


In [52]:
pages = 50;
movie_code = [189150, 191920, 36843, 195694, 194644, 187310, 39614];

mscore = [];
mtext = [];

print('Strat....')
for m in range(len(movie_code)):
    print('Movie Code:', movie_code[m]);
    for p in range(1, pages):
        print('start '+str(p)+'Page...');
        html = requests.get('https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code='+str(movie_code[m])+
                            '&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page='+str(p))
        # movie가 m, page가 p
        html_source = BeautifulSoup(html.content, 'html.parser');
        i = 0;
        for li in html_source.find('div', {'class': 'score_result'}).find_all(('li')):
            reviews = html_source.find( 'span', {'id':'_filtered_ment_' + str(i)});
            if int(li.em.text) > 8:
                mscore.append(1);
                mtext.append(reviews.text.strip());
            elif int(li.em.text) < 5:
                mscore.append(0);
                mtext.append(reviews.text.strip());
            i += 1;
print('End....')

Strat....
Movie Code: 189150
start 1Page...
start 2Page...
start 3Page...
start 4Page...
start 5Page...
start 6Page...
start 7Page...
start 8Page...
start 9Page...
start 10Page...
start 11Page...
start 12Page...
start 13Page...
start 14Page...
start 15Page...
start 16Page...
start 17Page...
start 18Page...
start 19Page...
start 20Page...
start 21Page...
start 22Page...
start 23Page...
start 24Page...
start 25Page...
start 26Page...
start 27Page...
start 28Page...
start 29Page...
start 30Page...
start 31Page...
start 32Page...
start 33Page...
start 34Page...
start 35Page...
start 36Page...
start 37Page...
start 38Page...
start 39Page...
start 40Page...
start 41Page...
start 42Page...
start 43Page...
start 44Page...
start 45Page...
start 46Page...
start 47Page...
start 48Page...
start 49Page...
Movie Code: 191920
start 1Page...
start 2Page...
start 3Page...
start 4Page...
start 5Page...
start 6Page...
start 7Page...
start 8Page...
start 9Page...
start 10Page...
start 11Page...
start 12Pa

## 2. 수집된 TEXT를 파일로 저장한다.

In [53]:
df = pd.DataFrame([mscore, mtext]).T;
df.columns = ['score','text'];
df['score'].value_counts();
df.to_excel('movies.xlsx', encoding='utf-8-sig');



## 3. 파일로 저장한 TEXT를 가지고, koNLPy를 이용하여 명사 단어를 추출한다.

In [54]:
okt = Okt();
mdata = pd.read_excel('movies.xlsx')
mdata.head(10)

,Unnamed: 0,score,text
0,0,1,뇌를 비우고 즐기면 된다
1,1,1,시리즈 팬들에게 주는 선물 같았던 엔딩..크.... 아련아련 ㅜㅜ
2,2,1,역시 분노의 질주! 이런 블록버스터는 역시 극장에서 봐야지
3,3,1,와 오랜만에 극장에서 봤는데 스케일 도랐 액션 개쩜
4,4,1,티라노사우루스는 팔이 짧아 분노의 질주 꿀잼 액션을 보고도 박수를 못쳐서 멸종됐다.
5,5,1,전작까지는 우주빼고 안가본데가 없었는데 이제 우주까지 나가네요..
6,6,1,졸라 신난다 진짜ㅋㅋㅋ 아 아침 일찍 보고 왔는데 보람있군
7,7,1,저스틴 린 감독이 진짜 오리지널 감독이긴 하네. 진짜 <분노의 질주> 시리즈 보는 ...
8,8,1,진짜 상상이상의 액션ㅋㅋㅋㅋ 아직 안본 분들 정말 상상도 못하셨을 겁니다
9,9,1,너무 재미있었어요~~ 기다린 보람이 있었어요 다음편두 더 기대할게요


In [55]:
len(mdata)

2494

## 4. 학습 텍스트와 테스트 텍스트로 나눈다.

In [56]:
import re

mdata = mdata[mdata['text'].notnull()];
mdata['text'] = mdata['text'].apply(lambda x:re.sub(r'[^ ㄱ-ㅣ가-힣 ]+', " ", x));
text = mdata['text'];
score = mdata['score'];



from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    text, score, test_size=0.2, random_state=777
);

len(x_train), len(x_test), len(y_train), len(y_test)

(1908, 478, 1908, 478)

## 5.  TfidfVectorizer를 이용하여 수치화 한다.

In [57]:
import warnings

warnings.filterwarnings('ignore');

from sklearn.feature_extraction.text import TfidfVectorizer


tfv = TfidfVectorizer(tokenizer=okt.morphs, min_df=3, max_df=0.9);

print('Row Data..');
print(x_train);
print(y_train);
print('TfidfVectorizer Data..')
tfv.fit(x_train);
tfv_x_train = tfv.transform(x_train);
print(tfv_x_train);

Row Data..
1357                             아  발간세상에선 이런거도 영화라고 하는구나
2463                           반전을 알고 봤는데도 여전히 소름돋는 충격   
1147    성기형 간만에 작품이라 봤는데 정치적인 문제 역사적인 문제를 떠나서 너무 어설프다 ...
1354                        대한민국 민주주의 역사의수도나 다름없는  월의 광주 
2474                            솔직히 징그럽습니다ㅠ반전에반전  정말머싰습니다
                              ...                        
1048    봐도 봐도 좋은 영화세상의 아직은 따스한 모습들을 모아 보여줘서 차가운 날씨마저 따...
117                                스토리가 산으로 가다못해 우주로 가버렸다
1788    진심 영화 본 뒤에 내 이목구비가 전부 간지러운 듯한 느낌은 처음이었음  예지 저렇...
2221     지금껏봐온반전중최고 시리즈중최고이며 내가본공포영화중에서도세손가락안에드는수작 결말을...
2254                     그래도  까지는 괜찮았던것 같은데 이게 무려  년도 영화임
Name: text, Length: 1908, dtype: object
1357    0
2463    1
1147    0
1354    1
2474    1
       ..
1048    1
117     0
1788    1
2221    1
2254    1
Name: score, Length: 1908, dtype: int64
TfidfVectorizer Data..
  (0, 1141)	0.33428246959919955
  (0, 1030)	0.17918326663148376
  (0, 990)	0.5433458839622524
  (0, 848)	0.36670121404093753
  (0, 765)	0.

## 6. 머신러닝을 이용하여 학습한다.
> 이진분류

In [58]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# 하이퍼 파라미터 가중치 1~22의 값을 넣어줌.
params = {'C':[5,6,9,11,15,18, 20, 22]}
lr = LogisticRegression();
grid_cv = GridSearchCV(lr,param_grid=params, cv=3, scoring='accuracy', verbose=0)

print('train ...')
grid_cv.fit(tfv_x_train, y_train);

print('End Train ...')
print(grid_cv.best_params_, grid_cv.best_score_)

train ...
End Train ...
{'C': 6} 0.8333333333333334


#### `grid_cv.best_params_` : 5라는 하이퍼파라미터가 가장 좋았다.

## 7.TEST 진행

In [59]:
from sklearn.metrics import accuracy_score

tfv_x_test = tfv.transform(x_test);
test_pr = grid_cv.best_estimator_.predict(tfv_x_test);
print('정확도 :', accuracy_score(y_test, test_pr))

정확도 : 0.8493723849372385


## 8. 실제 입력한 문장을 기반으로 예측

In [61]:
input_text = '시간 낭비한 영화 이런 영화는 보기 싫어';
input_text = re.compile(r'[ ㄱ-ㅣ가-힣]+').findall(input_text)
input_text = ['  '.join(input_text)];

tfv_input_text = tfv.transform(input_text);
# print(tfv_input_text)

result = grid_cv.best_estimator_.predict(tfv_input_text)
print('result:',result)
if result == 0:
    print('예측결과 :  -->> 부정')
else:
    print('예측결과 :  -->> 긍정')

result: [0]
예측결과 :  -->> 부정
